In [1]:
import sys
sys.path.insert(0, "/Users/rdeits/locomotion/drake-distro/rd-swig-matlab/build/lib/python2.7/dist-packages")

In [2]:
import numpy as np
import pydrake
from pydrake.solvers import ik
from pydrake.examples.pendulum import PendulumWithBotVis

/Users/rdeits/locomotion/drake-distro/rd-swig-matlab/build/lib/python2.7/dist-packages/pydrake/__init__.py:4: FutureWarning: This API is completely experimental and likely to change very soon. Use with caution.
  warnings.warn("This API is completely experimental and likely to change very soon. Use with caution.", FutureWarning)


In [3]:
# Construct our model
r = pydrake.rbm.RigidBodyManipulator("../../examples/Pendulum/Pendulum.urdf")
lc = pydrake.wrappers.lcm.LCM()
p = PendulumWithBotVis(lc)

In [4]:
# Inverse Kinematics:
# 
# Construct a posture constraint for the pendulum's joint
q = -0.9
posture_constraint = ik.PostureConstraint(r)
posture_constraint.setJointLimits(np.array([[6]], dtype=np.int32), 
                                  np.array([[q]]), 
                                  np.array([[q]]))
# Choose a seed configuration (randomly) and a nominal configuration (at 0)
q_seed = np.vstack((np.zeros((6,1)), 
                    p.getRandomState()[:1]))
q_nom = np.vstack((np.zeros((6,1)), 
                   0.))

# Run the ik solver and draw the result
options = ik.IKoptions(r)
results = pydrake.solvers.ik.inverseKinSimple(r, 
                              q_seed, 
                              q_nom, 
                              [posture_constraint], 
                              options)
p.output(0., np.vstack((results.q_sol[6:], 0.)), np.zeros((1,)))

array([[-0.9],
       [ 0. ]])

In [12]:
# Kinematics and Gradients:
# 
# Pre-compute kinematic information
kinsol = r.doKinematics(results.q_sol, np.zeros_like(results.q_sol), 1, True)

In [13]:
kinsol

<pydrake.rbm.KinematicsCache_d; proxy of <Swig Object of type 'KinematicsCache< double > *' at 0x10f256720> >

In [14]:
# Compute the center of mass
r.centerOfMass(kinsol)

array([[ 0.19583173],
       [ 0.        ],
       [-0.14790249]])

In [15]:
r.forwardKin(kinsol, np.array([1.,0,1.]), 0, 2, 1)

array([[ 1.40493688],
       [ 0.        ],
       [-0.16171694],
       [ 0.        ],
       [ 0.9       ],
       [ 0.        ]])